# Prüfungsleitung Maschnielles Lernen: Gruppe 8

Isolet download: https://datahub.io/machine-learning/isolet

### Aufgabe 1

#### 1) Data Loading and Splitting 

In [3]:
# Import isolet.csv File
import pandas as pd

# import
df_raw_isolet = pd.read_csv("./raw_isolet.csv", delimiter=",")

# changing column name 'class' to 'letter' 'cause class is a reservated keyword in python 
df_raw_isolet.rename(columns={'class': 'letter'}, inplace=True)
# removing char ' from letter
def removeApostrophes(value):
    if isinstance(value, str) and str(value).startswith('\'') and str(value).endswith('\''):
        return str(value).replace('\'', '')

df_raw_isolet.loc[:, 'letter'] = df_raw_isolet.letter.apply(lambda x: removeApostrophes(x))

##### Einfache Datenbetrachtung

In [4]:
print("Einträge in isolet.csv", len(df_raw_isolet))
print("Gruppierte Einträge:", df_raw_isolet.groupby(['letter']).size())

Einträge in isolet.csv 7797
Gruppierte Einträge: letter
1     300
10    300
11    300
12    300
13    299
14    300
15    300
16    300
17    300
18    300
19    300
2     300
20    300
21    300
22    300
23    300
24    300
25    300
26    300
3     300
4     300
5     300
6     298
7     300
8     300
9     300
dtype: int64


Bei der Betrachtung der Anzahl ist aufällig, dass in der Regel pro Buchstabe 300 Einträge aufgezeichnet wurden. Da aber len(df) nur 7797 Einträge beinhaltet, gibt es einige Ausnahmen. <br>
Ausnahmen: Buchstabe 13 mit 299 Einträgen und Buchstabe 6 mit 298 Einträgen <br><br>
Insgesamt bietet die sehr ausgewogene Verteilung der jeweiligen Buchstaben eine sehr gute Basis dies in dem Data Splitting zu betrachten. So wird für jeden Buchstaben ein Testsatz von je 75 Einträhge bestimmt. Formel: ⎡Anzahl Einträge pro Buchstabe * 0,25⎤

##### Data Splitting

In [55]:
df_splitting_training = []
df_splitting_test = []
numberOfTestEntries = 75

i = 0
for i in range(26):
    df_splitting_training.append(df_raw_isolet[df_raw_isolet.letter.str.match("^" + str(i+1) + "$")])

# test case that each enry has a max of 300 entries
for letterIndex, df in enumerate(df_splitting_training):
    if len(df_splitting_training[letterIndex]) > 300:
        print("WARNING: df_splitting[" + str(letterIndex) + "] has more than 300 entries!")

# split into test and training sets 
for letterIndex, df in enumerate(df_splitting_training):
    df_splitting_test.append(df_splitting_training[letterIndex].head(numberOfTestEntries))
    df_splitting_training[letterIndex].drop(index=df_splitting_training[letterIndex].index[:numberOfTestEntries], inplace=True)


C:\Users\Kev\AppData\Local\Temp\ipykernel_19804\503049006.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_splitting_training[letterIndex].drop(index=df_splitting_training[letterIndex].index[:numberOfTestEntries], inplace=True)


#### 2) Merkmalsreduktion

#### 3) Implementierung der Klassifikatoren

#### 4) Evaluation